In [1]:
# Import all the required files
import pureml
from pureml.decorators import load_data,transformer,dataset,model
import numpy as np
import pandas as pd
import lightgbm as lgb
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
import warnings
import random

warnings.simplefilter("ignore")
rand_seed = 1234
np.random.seed(rand_seed)

In [2]:
@load_data()
def load_dataset():
    # df = pd.read_excel(io='default of credit card clients.xls',header  = 1)
    df = pd.read_csv('default of credit card clients.csv', header=1)

    return df

In [3]:
@transformer()
def remove_columns(df):
    return df.drop(['ID'],axis =1)

In [4]:
@transformer()
def rename_columns(df):
    return df.rename(columns={"PAY_0": "PAY_1","default payment next month":"default", "SEX":"sex"})    

In [5]:
@transformer()
def dataset_imbalances(df):
    categorical_features = ["sex", "EDUCATION", "MARRIAGE"]

    for col_name in categorical_features:
        df[col_name] = df[col_name].astype("category")

    Y, A = df.loc[:, "default"], df.loc[:, "sex"]
    X = pd.get_dummies(df.drop(columns=["default", "sex"]))


    A_str = A.map({1: "male", 2: "female"})

    A_str.value_counts(normalize=True)
    Y.value_counts(normalize=True)
    
    # Generate "Interest" column as a DataFrame
    interest_values = np.random.normal(loc=2 * Y, scale=A)
    interest_column = pd.DataFrame(interest_values, columns=["Interest"])

    # Concatenate "Interest" column with X DataFrame
    X = pd.concat([X, interest_column], axis=1)

    return {'X':X,'Y':Y,'A_str':A_str}


In [6]:
@transformer()
def resample_training_data(X_train, Y_train, A_train):
   
    negative_ids = Y_train[Y_train == 0].index
    positive_ids = Y_train[Y_train == 1].index
    balanced_ids = positive_ids.union(
        np.random.choice(a=negative_ids, size=len(positive_ids)))

    X_train = X_train.loc[balanced_ids, :]
    Y_train = Y_train.loc[balanced_ids]
    A_train = A_train.loc[balanced_ids]
    return  {"X_train": X_train, "Y_train":Y_train, "A_train": A_train}



In [7]:

@transformer()
def add_new_column(sensitive_features):
    values = ['Indian', 'African', 'American']

    list_length = sensitive_features.shape[0]
    full_list = values * (list_length // len(values))
    full_list += values[:list_length % len(values)]
    random.shuffle(full_list)

    full_list = np.array(full_list)

    s_feat = pd.concat([sensitive_features.reset_index(drop=True), pd.DataFrame(full_list, columns=['race'])], axis=1)

    return s_feat

In [8]:
@dataset(label='temp_123_3',upload=True)
def create_dataset():
    df = load_dataset()
    df = remove_columns(df)
    df = rename_columns(df)
    data  = dataset_imbalances(df)
    X,Y,A_str = data['X'],data['Y'],data['A_str']
    X_train, X_test, y_train, y_test, A_train, A_test = train_test_split(X, Y, A_str, test_size=0.35, stratify=Y)
    data = resample_training_data(X_train, y_train, A_train)
    X_train, y_train, A_train = data['X_train'],data['Y_train'],data['A_train']

    A_test = add_new_column(sensitive_features=A_test)

    return {"x_train":X_train,"y_train":y_train.to_numpy(),"x_test":X_test,"y_test":y_test.to_numpy(),"sensitive_features" : A_test}


data_created = create_dataset()

Dataset details have not been found

Dataset has been created!

Hash :  fdef4116272b4d04a0ed1e19144d10f7

Version:  v1

Dataset label:  temp_123_3:v1

Dataset and lineage have been registered!

In [9]:
print(data_created["sensitive_features"].shape)
print(data_created["x_test"].shape)
print(data_created["y_test"].shape)

(10500, 2)
(10500, 32)
(10500,)


In [10]:
@model(label='temp_model_123_3')
def create_model():
    data = pureml.dataset.fetch('temp_123_3:v1')
    x_train = data['x_train']
    y_train = data['y_train']
    lgb_params = {
    "objective": "binary",
    "metric": "auc",
    "learning_rate": 0.412,
    "num_leaves": 10,
    "max_depth": 3,
    "random_state": rand_seed,
    "n_jobs": 1,}

    pureml.log(params=lgb_params)
    estimator = Pipeline(
        steps=[
            ("preprocessing", StandardScaler()),
            ("classifier", lgb.LGBMClassifier(**lgb_params)),
        ]
    )

    estimator.fit(x_train, y_train)
    return estimator

model_lgb = create_model()

[LightGBM] [Info] Number of positive: 4313, number of negative: 4313
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.002328 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 3523
[LightGBM] [Info] Number of data points in the train set: 8626, number of used features: 29
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.500000 -> initscore=0.000000
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positi

Model details have not been found

Model has been created!

model_created True

Model has been registered!

Model Version:  v1

Model label:  temp_model_123_3:v1

No metrics are found in config


Params have been registered!

No figures are found in config


In [11]:
pureml.predict.add(label='temp_model_123_3:v1',paths={'predict':'predict.py'})

Predict Function has been registered!

{"status":200,"data":[{"key":"predict","data":"pureml_data/predict.py","model_version":{"uuid":"5228b90d-ad8d-4446-
a861-e09cc6224ee1","version":"v1"},"dataset_version":{"uuid":"00000000-0000-0000-0000-000000000000","version":""}}]
,"message":"Logs created"}

In [12]:
import pureml_policy
pureml_policy.__version__

'0.2.2'

In [14]:
from pureml_policy import policy_eval
#policy = faircredit
results = policy_eval.eval(framework_name  = "nyc144",
            label_model='temp_model_123_3:v1',
            label_dataset='temp_123_3:v1')

 Succesfully fetched the dataset

 Succesfully fetched the predictor

 Succesfully fetched the model

Dataset Keys: dict_keys(['x_train', 'y_train', 'x_test', 'y_test', 'sensitive_features'])

'balanced_accuracy'
argument of type 'NoneType' is not iterable
'balanced_acc_error'
argument of type 'NoneType' is not iterable
'disparate_impact'
argument of type 'NoneType' is not iterable
Unable to compute disparate_impact
too many indices for array: array is 1-dimensional, but 2 were indexed
Exception : list indices must be integers or slices, not str
'demographic_parity_difference'
argument of type 'NoneType' is not iterable
'balanced_accuracy'
argument of type 'NoneType' is not iterable
'balanced_acc_error'
argument of type 'NoneType' is not iterable
'disparate_impact'
argument of type 'NoneType' is not iterable
Unable to compute disparate_impact
'<' not supported between instances of 'dict' and 'dict'
Exception : list indices must be integers or slices, not str
'demographic_parity_difference'
argument of type 'NoneType' is not iterable
Unable to compute demographic_parity_difference
Feature lists must be of scalar types
Exception : list indices must be integers or slices, not s

In [16]:
results

{'model': 'temp_model_123_3',
 'model_version': 'v1',
 'dataset': 'temp_123_3',
 'dataset_version': 'v1',
 'result': [{'complete': {'complete': {'operational': {'accuracy': 'pass',
      'precision': 'pass',
      'recall': 'fail',
      'f1': 'fail'},
     'fairness': {'balanced_accuracy': 'fail',
      'balanced_acc_error': 'fail',
      'demographic_parity_difference': 'fail'},
     'operational_scores': {'accuracy': '0.7902857142857143',
      'precision': '0.8011192423590185',
      'recall': '0.5168008886420439',
      'f1': '0.6282916948008103'},
     'fairness_scores': {'balanced_accuracy': '0.7941636324305794',
      'balanced_acc_error': '0.009249067740303738',
      'demographic_parity_difference': '0.05995385011830301'},
     'operational_thresholds': {'accuracy': '0.7',
      'precision': '0.8',
      'recall': '0.8',
      'f1': '0.7'},
     'fairness_thresholds': {'balanced_accuracy': '0.8',
      'balanced_acc_error': '0.7',
      'demographic_parity_difference': '0.8'}

In [17]:
results['result'][0]['subsets'][(('sex','male'),('race','American'))]

{'operational': {'accuracy': 'pass',
  'precision': 'pass',
  'recall': 'fail',
  'f1': 'pass'},
 'fairness': {'balanced_accuracy': 'pass', 'balanced_acc_error': 'fail'},
 'operational_scores': {'accuracy': '0.8390804597701149',
  'precision': '0.8393939393939394',
  'recall': '0.6183035714285714',
  'f1': '0.712082262210797'},
 'fairness_scores': {'balanced_accuracy': '0.8391884951206985',
  'balanced_acc_error': '0.022682804847256332'},
 'operational_thresholds': {'accuracy': '0.7',
  'precision': '0.8',
  'recall': '0.8',
  'f1': '0.7'},
 'fairness_thresholds': {'balanced_accuracy': '0.8',
  'balanced_acc_error': '0.7'}}